--> torch hook is great way to check the output of any network layer <br>
**Pros:** <br>
The torch hook can be used during inference. Any network (previously trained) can be loaded and hook ables to debug/print/visualize output during inference/test. <br>
**Cons:** <br>
hook can not print the intermediate variables in forward method.<br>

To get the intermediate value, it has to be encapsulated in another class

In [13]:
import torch
import torch.nn as nn
from collections import OrderedDict

In [62]:
class Matmul(nn.Module):
    def forward(self, **kwargs):
        return kwargs['op'] * kwargs['multiplier']

class DummyHookNet(nn.Module):
    def __init__(self, conf):
        super().__init__()
        
        self.dummy_linear_layer = nn.Sequential(OrderedDict([
            ('1_LINEAR_LAYER', nn.Linear(conf['in'], conf['out'])),
            ('2_MUL_LAYER', Matmul()),
            ('3_RELU_LAYER', nn.LeakyReLU()),
            ('4_DROP_LAYER', nn.Dropout(p=conf['drop_out']))
        ]))
        
    def forward(self, x, multiplier):
        layer_1 = self.dummy_linear_layer._modules['1_LINEAR_LAYER']
        layer_1_op = layer_1(x)
        
        layer_2 = self.dummy_linear_layer._modules['2_MUL_LAYER']
        layer_2_op = layer_2(op = layer_1_op, multiplier = multiplier)
        
        print("layer_2\n",layer_2_op)
        
        layer_3 = self.dummy_linear_layer._modules['3_RELU_LAYER']
        layer_3_op = layer_3(layer_2_op)
        
        layer_4 = self.dummy_linear_layer._modules['4_DROP_LAYER']
        layer_4_op = layer_4(layer_3_op)
        
        return layer_3_op

In [63]:
def invoke_hook():
    try:
        conf = {
            "in": 3,
            "out": 5,
            "drop_out": 0.1
        }
        dummy_hook_net = DummyHookNet(conf)
 
        dummy_handle_1 = dummy_hook_net.dummy_linear_layer._modules['1_LINEAR_LAYER'].register_forward_hook(
            lambda m,
            input,
            output: print("layer_1\n",output)
        )
    
        dummy_handle_2 = dummy_hook_net.dummy_linear_layer._modules['2_MUL_LAYER'].register_forward_hook(
            lambda m,
            input,
            output: print("layer_2\n",output)
        )
        
        inp = torch.rand(2, 3)
        oup = dummy_hook_net(inp, 5)
        # print(oup)
        
        dummy_handle_1.remove()
        dummy_handle_2.remove()
        
    except Exception as e:
        raise e

In [64]:
invoke_hook()

layer_1
 tensor([[ 0.3367, -0.0996,  0.7458,  0.0084,  0.0412],
        [ 0.3578, -0.1920,  0.6517, -0.1853,  0.1640]],
       grad_fn=<AddmmBackward0>)
layer_2
 tensor([[ 1.6837, -0.4979,  3.7288,  0.0421,  0.2061],
        [ 1.7888, -0.9600,  3.2583, -0.9267,  0.8202]], grad_fn=<MulBackward0>)
layer_2
 tensor([[ 1.6837, -0.4979,  3.7288,  0.0421,  0.2061],
        [ 1.7888, -0.9600,  3.2583, -0.9267,  0.8202]], grad_fn=<MulBackward0>)


In [ ]:
### Resources
1) https://dev.to/jankrepl/forward-hooks-in-pytorch-4eeh
2) https://github.com/elliotwaite/pytorch-hooks-tutorial
3) https://discuss.pytorch.org/t/how-can-l-load-my-best-model-as-a-feature-extractor-evaluator/17254/20